In [2]:
import os
import pandas as pd
import regex as re
import textwrap
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
cwd = os.getcwd()
print(cwd)

c:\Users\Blanca\Dropbox\My PC (Blanca)\Desktop\DSDM\brushup_files


### 1. EDA on text data

In [10]:
df_text = pd.read_csv(cwd + '/data/text_data.csv')
print(df_text.shape)
df_text.head()

(42, 3)


,country,title,event_type
0,KHM,President Expands Control: Courts Declare Oppo...,judiciary_weakened
1,TUR,Emergency Powers Extended: Constitutional Cour...,judiciary_weakened
2,PHL,Supreme Court Backs Martial Law Extension: Cri...,judiciary_weakened
3,ZWE,High Court Deemed 'Enemy of the State': Contro...,judiciary_weakened
4,VEN,Court Overhaul Clears Path for Sweeping Econom...,judiciary_weakened


In [11]:
# Function to wrap text of a specific cell
def wrap_text(text, width=50):
    """Wrap text to the specified width."""
    return textwrap.fill(text, width)

# Maximum width for line breaks
max_width = 130

# Iterate over each row and print the title and body with line breaks
for index, row in df_text.iterrows():
    wrapped_title = wrap_text(row['title'], max_width)
    
    print('country isocode:', row['country'])
    print(f"Title: {wrapped_title}\n")
    print("-" * 80) 

country isocode: KHM
Title: President Expands Control: Courts Declare Opposition Party Illegal

--------------------------------------------------------------------------------
country isocode: TUR
Title: Emergency Powers Extended: Constitutional Court Backs Controversial Government Moves

--------------------------------------------------------------------------------
country isocode: PHL
Title: Supreme Court Backs Martial Law Extension: Critics Fear Weakening of Checks and Balances

--------------------------------------------------------------------------------
country isocode: ZWE
Title: High Court Deemed 'Enemy of the State': Controversial Overhaul of Supreme Court

--------------------------------------------------------------------------------
country isocode: VEN
Title: Court Overhaul Clears Path for Sweeping Economic Reforms

--------------------------------------------------------------------------------
country isocode: VEN
Title: Judiciary Under Fire: Judges Resign as Gover

In [5]:
df_text['keywords'] = df_text['title'].str.findall(r'\b[A-Z][a-z]+\b')

df_text['contains_judge'] = df_text['title'].str.contains(r'\b(Judge|Judges)\b', regex=True)

df_text['bigrams'] = df_text['title'].str.findall(r'\b\w+\b \b\w+\b')

df_text['word_count_1'] = df_text['title'].str.count(r'\b\w+\b')

df_text['word_count_2'] = df_text['title'].str.split().str.len()

df_text.head(3)


C:\Users\Blanca\AppData\Local\Temp\ipykernel_4436\1648143037.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_text['contains_judge'] = df_text['title'].str.contains(r'\b(Judge|Judges)\b', regex=True)


,country,title,event_type,keywords,contains_judge,bigrams,word_count_1,word_count_2
0,KHM,President Expands Control: Courts Declare Oppo...,judiciary_weakened,"[President, Expands, Control, Courts, Declare,...",False,"[President Expands, Courts Declare, Opposition...",8,8
1,TUR,Emergency Powers Extended: Constitutional Cour...,judiciary_weakened,"[Emergency, Powers, Extended, Constitutional, ...",False,"[Emergency Powers, Constitutional Court, Backs...",9,9
2,PHL,Supreme Court Backs Martial Law Extension: Cri...,judiciary_weakened,"[Supreme, Court, Backs, Martial, Law, Extensio...",False,"[Supreme Court, Backs Martial, Law Extension, ...",13,13


In [6]:
my_sentence = "Hello, how are you? I'm ok thanks. How are you?"

list_of_words = my_sentence.split()
print(list_of_words)
re.findall(r'\b[A-Z][a-z]+\b', my_sentence)

['Hello,', 'how', 'are', 'you?', "I'm", 'ok', 'thanks.', 'How', 'are', 'you?']


['Hello', 'How']

In [7]:
df_text.__contains__(r'Hello', regex=True)

TypeError: NDFrame.__contains__() got an unexpected keyword argument 'regex'

### 2. Combining data sets

Key Differences:
1. Merge combines two DataFrames based on one or more common columns.
2. Concatenate appends DataFrames along a particular axis.
3. Join combines DataFrames based on their index values.


In [12]:
# example DataFrames
df1 = pd.DataFrame({
    'employee_id': [101, 102, 103],
    'name': ['Alice', 'Bob', 'Charlie']
})

df2 = pd.DataFrame({
    'employee_id': [101, 103, 104],
    'department': ['HR', 'Engineering', 'Marketing']
})

df3 = pd.DataFrame({
    'name': ['David', 'Eva'],
    'department': ['Finance', 'HR']
})

df4 = pd.DataFrame({
    'salary': [70000, 80000, 60000]
})

# two with the same index
df5 = pd.DataFrame({
    'age': [25, 30, 22],
}, index=['Alice', 'Bob', 'Charlie'])

df6 = pd.DataFrame({
    'salary': [50000, 60000, 55000]
}, index=['Alice', 'Bob', 'Charlie'])

#### Merging

Experiment with the difference the "how" parameter makes:

Your options: how{‘left’, ‘right’, ‘outer’, ‘inner’, ‘cross’}, default ‘inner’

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html

In [32]:
print(df1)
print(df2)

   employee_id     name
0          101    Alice
1          102      Bob
2          103  Charlie
   employee_id   department
0          101           HR
1          103  Engineering
2          104    Marketing


In [33]:
# Merge on 'employee_id'
merged_df = pd.merge(df1, df2, on='employee_id', how = 'inner')
print("Merged DataFrame:")
merged_df

Merged DataFrame:


,employee_id,name,department
0,101,Alice,HR
1,103,Charlie,Engineering


In [52]:
complete_df = pd.merge(df1, df2, on='employee_id', how='left')
complete_df

,employee_id,name,department
0,101,Alice,HR
1,102,Bob,NaN
2,103,Charlie,Engineering


#### Concatenating

In [34]:
print(df1)
print(df3)

   employee_id     name
0          101    Alice
1          102      Bob
2          103  Charlie
    name department
0  David    Finance
1    Eva         HR


In [35]:
# Concatenate two DataFrames along rows (axis=0)
concatenated_df = pd.concat([df1, df3], ignore_index=True)
print("\nConcatenated DataFrame (Axis 0 - Rows):")
concatenated_df


Concatenated DataFrame (Axis 0 - Rows):


,employee_id,name,department
0,101.0,Alice,NaN
1,102.0,Bob,NaN
2,103.0,Charlie,NaN
3,NaN,David,Finance
4,NaN,Eva,HR


In [48]:
print(df1)
print(df4)

   employee_id     name
0          101    Alice
1          102      Bob
2          103  Charlie
   salary
0   70000
1   80000
2   60000


In [37]:
concatenated_columns_df = pd.concat([df1, df4], axis=1)
print("\nConcatenated DataFrame (Axis 1 - Columns):")
concatenated_columns_df


Concatenated DataFrame (Axis 1 - Columns):


,employee_id,name,salary
0,101,Alice,70000
1,102,Bob,80000
2,103,Charlie,60000


#### Joining

In [49]:
print(df5)
print(df6)

         age
Alice     25
Bob       30
Charlie   22
         salary
Alice     50000
Bob       60000
Charlie   55000


In [39]:
# Join the two DataFrames
joined_df = df5.join(df6)
print("Joined DataFrame (Using Index):")
joined_df

Joined DataFrame (Using Index):


,age,salary
Alice,25,50000
Bob,30,60000
Charlie,22,55000


### 3. More on WB data

#### 3.0 Data

In [53]:
df_WB_more_data = pd.read_csv(cwd + '/data/WB_more_data.csv')

In [41]:
# copy the data
df_og = df_WB_more_data.copy()

# check what series and countries are included
print(df_og['Series Name'].unique())
print(df_og['Country Name'].nunique())
df_og.sample(3)

['Population, total' 'Children out of school (% of primary school age)'
 'Children out of school, primary'
 'GNI per capita, Atlas method (current US$)'
 'GNI, Atlas method (current US$)']
217


,Series Name,Series Code,Country Name,Country Code,2001,2002,2003,2011,2012,2013,2021,2022,2023
374,Children out of school (% of primary school age),SE.PRM.UNER.ZS,Portugal,PRT,0.2714599967,0.01350000035,0.07801999897,0.00935999956,0.192839995,0.126819998,0.07241000235,..,..
640,"Children out of school, primary",SE.PRM.UNER,United States,USA,..,..,..,..,..,711515,996160,965853,..
878,"GNI, Atlas method (current US$)",NY.GNP.ATLS.CD,Australia,AUS,391827628839,393994724140,421653243724,1126187841984,1370648411464,1528597111561,1470330517618,1582213831308,1681999462379


In [54]:
df_og.describe()

,Series Name,Series Code,Country Name,Country Code,2001,2002,2003,2011,2012,2013,2021,2022,2023
count,1085,1085,1085,1085,1085,1085,1085,1085,1085,1085,1085,1085,1085
unique,5,5,217,217,797,813,810,875,882,872,869,792,597
top,"Population, total",SP.POP.TOTL,Afghanistan,AFG,..,..,..,..,..,..,..,..,..
freq,217,217,5,5,252,240,242,200,190,206,202,287,477


#### 3.1 Melting and pivoting

In [43]:
# melt the data frame 
year_cols = ['2001', '2002', '2003', '2011', '2012', '2013', '2021', '2022', '2023']

df_melted = pd.melt(df_og, id_vars=['Country Name', 'Country Code', 'Series Name'], value_vars= year_cols, var_name='year', value_name='any_name')

df_melted.sample(5)


,Country Name,Country Code,Series Name,year,any_name
5264,Dominican Republic,DOM,"GNI, Atlas method (current US$)",2012,57491440475
2635,Burundi,BDI,"Children out of school, primary",2003,508520
2676,Georgia,GEO,"Children out of school, primary",2003,..
411,Togo,TGO,Children out of school (% of primary school age),2001,12.54244995
7594,Zimbabwe,ZWE,"GNI, Atlas method (current US$)",2021,24553581682


In [44]:
## Solution
print(df_melted.shape)
print(df_melted['Series Name'].value_counts())
print(df_melted['year'].value_counts())

print(217*9)
print(217*5)

(9765, 5)
Series Name
Population, total                                   1953
Children out of school (% of primary school age)    1953
Children out of school, primary                     1953
GNI per capita, Atlas method (current US$)          1953
GNI, Atlas method (current US$)                     1953
Name: count, dtype: int64
year
2001    1085
2002    1085
2003    1085
2011    1085
2012    1085
2013    1085
2021    1085
2022    1085
2023    1085
Name: count, dtype: int64
1953
1085


In [45]:
# pivot the data frame
pivoted_df = df_melted.pivot(index=['Country Name', 'Country Code', 'year'], columns='Series Name', values='any_name').reset_index()
print(pivoted_df.shape)
pivoted_df.sample(4)

(1953, 8)


Series Name,Country Name,Country Code,year,Children out of school (% of primary school age),"Children out of school, primary","GNI per capita, Atlas method (current US$)","GNI, Atlas method (current US$)","Population, total"
168,Belgium,BEL,2021,1.474030018,11848,51060,591607807362,11586195
462,Cyprus,CYP,2011,0.1532399952,82,32680,27810261261,1145086
673,Ghana,GHA,2022,..,..,2380,79663089865,33475870
284,Burundi,BDI,2013,..,140041,240,2477670972,10149577


#### 3.2 Renaming & missingness

In [46]:
df = pivoted_df.copy()

In [65]:
# use renaming dictionary
rename_dict = {
    'Country Code' : 'isocode',
    'Country Name' : 'country',
    'Children out of school (% of primary school age)': 'primary_out_school_pct',
    'Children out of school, primary' : 'primary_out_school_abs',
    'GNI per capita, Atlas method (current US$)' : 'gni_percapita',
    'GNI, Atlas method (current US$)': 'gni_total',
    'Population, total': 'population_total'
}

df.rename(columns=rename_dict, inplace=True)

df.columns

Index(['country', 'isocode', 'year', 'primary_out_school_pct',
       'primary_out_school_abs', 'gni_percapita', 'gni_total',
       'population_total'],
      dtype='object', name='Series Name')

In [63]:
# round to two decimal places

df['primary_out_school_pct'] = round(df['primary_out_school_pct'],2)
df

Series Name,country,isocode,year,primary_out_school_pct,"Children out of school, primary",gni_percapita,gni_total,population_total
0,Afghanistan,AFG,2001,..,..,..,..,19688632
1,Afghanistan,AFG,2002,..,..,180,3855298264,21000256
2,Afghanistan,AFG,2003,..,..,190,4330114358,22645130
3,Afghanistan,AFG,2011,..,..,550,16062706809,29249157
4,Afghanistan,AFG,2012,..,..,640,19575474864,30466479
...,...,...,...,...,...,...,...,...
1948,Zimbabwe,ZWE,2012,5.137090206,132003,1120,14854556176,13265331
1949,Zimbabwe,ZWE,2013,6.345449924,163860,1200,16249407148,13555422
1950,Zimbabwe,ZWE,2021,6.272999763,189407,1540,24553581682,15993524
1951,Zimbabwe,ZWE,2022,5.586999893,171679,1720,28003360601,16320537


In [70]:
df.loc[df.isocode == 'AFG']

Series Name,country,isocode,year,primary_out_school_pct,primary_out_school_abs,gni_percapita,gni_total,population_total
0,Afghanistan,AFG,2001,..,..,..,..,19688632
1,Afghanistan,AFG,2002,..,..,180,3855298264,21000256
2,Afghanistan,AFG,2003,..,..,190,4330114358,22645130
3,Afghanistan,AFG,2011,..,..,550,16062706809,29249157
4,Afghanistan,AFG,2012,..,..,640,19575474864,30466479
5,Afghanistan,AFG,2013,..,..,670,21060159749,31541209
6,Afghanistan,AFG,2021,..,..,380,15387504468,40099462
7,Afghanistan,AFG,2022,..,..,360,14891705444,41128771
8,Afghanistan,AFG,2023,..,..,..,..,42239854


In [82]:
# inspect the missingness
import missigno as msno
msno.matrix(df)


ModuleNotFoundError: No module named 'missigno'

In [72]:
# what is going on?
print(df.loc[df.isocode == 'AFG']['primary_out_school_abs'])

0    ..
1    ..
2    ..
3    ..
4    ..
5    ..
6    ..
7    ..
8    ..
Name: primary_out_school_abs, dtype: object


In [74]:
# let's fix it
df.replace('..', np.nan, inplace=True)

df.isnull().sum()

Series Name
country                     0
isocode                     0
year                        0
primary_out_school_pct    942
primary_out_school_abs    937
gni_percapita             210
gni_total                 207
population_total            0
dtype: int64

In [78]:
# only keep the rows that don't have missing values

dropped = df.dropna()

# what are we inspecting here?
dropped.shape

(982, 8)

In [83]:
# sets are useful for finding differences!
diff_iso = set(df.country.unique()) - set(dropped.country.unique())
len(diff_iso)
print(diff_iso)

set()


In [84]:
dropped.to_csv(cwd + "/data/WB_reshaped_nomissing.csv")

#### 3.3 Groupby


In [79]:
df = dropped.copy()

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 982 entries, 9 to 1951
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   country                 982 non-null    object
 1   isocode                 982 non-null    object
 2   year                    982 non-null    object
 3   primary_out_school_pct  982 non-null    object
 4   primary_out_school_abs  982 non-null    object
 5   gni_percapita           982 non-null    object
 6   gni_total               982 non-null    object
 7   population_total        982 non-null    object
dtypes: object(8)
memory usage: 69.0+ KB


In [86]:
df = df.astype({'year':'int', 'primary_out_school_pct':'float','primary_out_school_abs':'float','gni_percapita':'float','gni_total':'float','population_total':'float' })

In [87]:
df.sample(4)

Series Name,country,isocode,year,primary_out_school_pct,primary_out_school_abs,gni_percapita,gni_total,population_total
201,Bhutan,BTN,2011,9.20584,9044.0,2380.0,1.696377e+09,713331.0
1685,Sweden,SWE,2003,0.34163,2457.0,32750.0,2.933868e+11,8958229.0
18,Algeria,DZA,2001,2.97668,129143.0,1730.0,5.403291e+10,31200985.0
1143,Mexico,MEX,2001,0.17107,23211.0,7150.0,7.111491e+11,99394288.0


In [89]:
# only now we can grouby

df.groupby(['country'])[['gni_percapita', 'gni_total']].agg(['min','mean','median','max'])

Series Name         gni_percapita                                  \
                              min          mean   median      max   
country                                                             
Albania                    1280.0   4158.571429   4410.0   6770.0   
Algeria                    1730.0   4012.222222   4490.0   6030.0   
Andorra                   44350.0  47215.000000  47215.0  50080.0   
Angola                     3410.0   3410.000000   3410.0   3410.0   
Antigua and Barbuda       14010.0  14190.000000  14280.0  14280.0   
...                           ...           ...      ...      ...   
Viet Nam                    400.0   1326.000000   1630.0   2200.0   
West Bank and Gaza         1190.0   2810.000000   3045.0   4720.0   
Yemen, Rep.                 450.0    870.000000    845.0   1340.0   
Zambia                      360.0   1102.000000   1390.0   1720.0   
Zimbabwe                    340.0    954.285714   1120.0   1720.0   

Series Name             gni_total                                            
                              min          mean        median           max  
country                                                                      
Albania              3.906030e+09  1.192744e+10  1.280394e+10  1.881509e+10  
Algeria              5.403291e+10  1.576934e+11  1.862424e+11  2.293214e+11  
Andorra              3.504812e+09  3.751327e+09  3.751327e+09  3.997843e+09  
Angola               8.260439e+10  8.260439e+10  8.260439e+10  8.260439e+10  
Antigua and Barbuda  1.214841e+09  1.243484e+09  1.252064e+09  1.263545e+09  
...                           ...           ...           ...           ...  
Viet Nam             3.218636e+10  1.172216e+11  1.442171e+11  1.988829e+11  
West Bank and Gaza   3.665299e+09  1.187296e+10  1.196730e+10  2.381074e+10  
Yemen, Rep.          8.683417e+09  2.150536e+10  2.059957e+10  3.613888e+10  
Zambia               3.691193e+09  1.564798e+10  1.988882e+10  2.626911e+10  
Zimbabwe             4.110330e+09  1.383830e+10  1.485456e+10  2.800336e+10  

[184 rows x 8 columns]

In [92]:
gni_df = df.groupby(['country'])['gni_percapita'].agg(['min','mean','median','max'])
gni_df.head(1)

,min,mean,median,max
country,,,,
Albania,1280.0,4158.571429,4410.0,6770.0


In [97]:
gni_df['difference'] = gni_df['max'] - gni_df['min']
gni_df['pct_difference'] = round(((gni_df['max'] - gni_df['min'])/gni_df['min'])*100,2)

gni_df.head()

,min,mean,median,max,difference,pct_difference
country,,,,,,
Albania,1280.0,4158.571429,4410.0,6770.0,5490.0,428.91
Algeria,1730.0,4012.222222,4490.0,6030.0,4300.0,248.55
Andorra,44350.0,47215.000000,47215.0,50080.0,5730.0,12.92
Angola,3410.0,3410.000000,3410.0,3410.0,0.0,0.00
Antigua and Barbuda,14010.0,14190.000000,14280.0,14280.0,270.0,1.93


#### 3.4 Features

In [98]:
df.describe()

Series Name,year,primary_out_school_pct,primary_out_school_abs,gni_percapita,gni_total,population_total
count,982.000000,982.000000,9.820000e+02,982.000000,9.820000e+02,9.820000e+02
mean,2010.738289,8.430316,2.484152e+05,13428.075356,3.104510e+11,3.208063e+07
std,7.485749,12.157289,9.541054e+05,18880.642122,1.368748e+12,1.251424e+08
min,2001.000000,0.000000,0.000000e+00,110.000000,7.652380e+07,1.044400e+04
25%,2003.000000,1.084265,2.386500e+03,1480.000000,5.803913e+09,2.059770e+06
50%,2012.000000,3.615655,1.927750e+04,5090.000000,2.647996e+10,7.934390e+06
75%,2013.000000,9.761242,1.052205e+05,17665.000000,1.679729e+11,2.265672e+07
max,2023.000000,70.010399,1.579330e+07,104830.000000,2.552691e+13,1.428628e+09


In [103]:
df['gni_total_bil'] = round(df['gni_total']/1_000_000_000, 3)
df.rename(columns={'population_total':'pop_total'},inplace=True)
df.rename(columns={'gni_total_bil':'gni_bil'},inplace=True)
df['pop_mil'] = round(df['pop_total']/1_000_000, 3)
df['gni_bil_pc'] = round(df['gni_percapita']/1_000_000_000, 3) 
# df['prim_oos_pct'] = round(df['gni_pc']/1_000_000_000, 3)

df.head()

Series Name,country,isocode,year,primary_out_school_pct,primary_out_school_abs,gni_percapita,gni_total,pop_total,gni_bil,pop_mil,gni_bil,gni_bil_pc
9,Albania,ALB,2001,1.43961,3630.0,1280.0,3.906030e+09,3060173.0,3.906,3.060,3.906,0.0
11,Albania,ALB,2003,1.93195,4554.0,1650.0,5.006223e+09,3039616.0,5.006,3.040,5.006,0.0
12,Albania,ALB,2011,0.16994,334.0,4410.0,1.280394e+10,2905195.0,12.804,2.905,12.804,0.0
13,Albania,ALB,2012,0.21698,405.0,4360.0,1.265829e+10,2900401.0,12.658,2.900,12.658,0.0
14,Albania,ALB,2013,0.37125,666.0,4540.0,1.315689e+10,2895092.0,13.157,2.895,13.157,0.0


In [108]:
df.drop('gni_bil_pc', axis=1)

Series Name,country,isocode,year,primary_out_school_pct,primary_out_school_abs,gni_percapita,gni_total,pop_total,gni_bil,pop_mil,gni_bil
9,Albania,ALB,2001,1.43961,3630.0,1280.0,3.906030e+09,3060173.0,3.906,3.060,3.906
11,Albania,ALB,2003,1.93195,4554.0,1650.0,5.006223e+09,3039616.0,5.006,3.040,5.006
12,Albania,ALB,2011,0.16994,334.0,4410.0,1.280394e+10,2905195.0,12.804,2.905,12.804
13,Albania,ALB,2012,0.21698,405.0,4360.0,1.265829e+10,2900401.0,12.658,2.900,12.658
14,Albania,ALB,2013,0.37125,666.0,4540.0,1.315689e+10,2895092.0,13.157,2.895,13.157
...,...,...,...,...,...,...,...,...,...,...,...
1946,Zimbabwe,ZWE,2003,5.86950,126586.0,340.0,4.110330e+09,12075828.0,4.110,12.076,4.110
1948,Zimbabwe,ZWE,2012,5.13709,132003.0,1120.0,1.485456e+10,13265331.0,14.855,13.265,14.855
1949,Zimbabwe,ZWE,2013,6.34545,163860.0,1200.0,1.624941e+10,13555422.0,16.249,13.555,16.249
1950,Zimbabwe,ZWE,2021,6.27300,189407.0,1540.0,2.455358e+10,15993524.0,24.554,15.994,24.554


Suppose you estimate that working-age population is roughly 60% of the total population. You can create a dependency ratio:

In [ ]:
df['primary_aged_total'] =
df['primary_aged_of_pop'] = 

# using appy

df['dependency'] = df.apply(
    lambda row: row['primary_out_school_total'] / (row['pop'] * 0.6), axis=1
)

df.sample(3)

In [220]:
# using a lambda function and mapping a dictionary

income_groups = {
    lambda x: x < 1045: 'low',
    lambda x: 1045 <= x < 4095: 'low_mid',
    lambda x: 4095 <= x < 12695: 'upp_mid',
    lambda x: x >= 12695: 'high'
}

df['income_group'] = df['gni_pc'].map(
    lambda x: next((v for k, v in income_groups.items() if k(x)), None)
)
df.income_group.value_counts()

#### 3.5 Extra

In [ ]:
df_highest_year = df.sort_values('???', ascending=False).drop_duplicates('???')
df_highest_year

In [ ]:
df_highest_year.boxplot(column='???')

In [ ]:
df_highest_year.gni_pc.nlargest(10)

In [ ]:
# show the box plot only for a given income group
df_highest_year.loc[???].boxplot(column='gni_pc')

In [ ]:
groups = ['high', 'upp_mid', 'low_mid', 'low']

# Create a figure with subplots
fig, axes = plt.subplots(1, len(groups), figsize=(12, 6))

# Loop through each group and create a boxplot in the corresponding subplot
for i, group in enumerate(groups):
    df_highest_year.loc[df_highest_year['income_group'] == ???].boxplot(column='gni_pc', ax=axes[???])
    axes[i].set_title(group)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# compare
df_highest_year.plot.scatter(x='???', y='???')

In [ ]:
# compare
df_highest_year.

In [ ]:
# Create a figure with subplots
fig, axes = plt.subplots(1, len(groups), figsize=(12, 6))

# Loop through each group and create a boxplot in the corresponding subplot
for i, group in enumerate(groups):
    ???
    axes[i].set_title(group)

# Adjust layout
plt.tight_layout()
plt.show()

### 4. Background cleaning

In [ ]:
df_WB = pd.read_csv(cwd + '/data/WB_full.csv')
#df_meta = pd.read_csv(cwd + '/data/WB_metadata.csv')

df_WB.head()

In [233]:
# clean column names
cols_to_clean = df_WB.columns.tolist()
rename_dict = {col: col.split()[0] for col in cols_to_clean if '[YR' in col}
df_WB = df_WB.rename(columns=rename_dict)

The net enrollment rate excludes overage and underage students and more accurately captures the system's coverage and internal efficiency. Differences between the gross enrollment ratio and the net enrollment rate show the incidence of overage and underage enrollments.
https://databank.worldbank.org/metadataglossary/world-development-indicators/series/SE.PRE.ENRR

In [ ]:
df_WB['Series Name'].unique()

In [235]:
series_to_keep = ['Population, total',
                  'GNI, Atlas method (current US$)', 'GNI per capita, Atlas method (current US$)',
                  'Children out of school, primary',
                  'Children out of school (% of primary school age)'
                  ]

In [236]:
df_choice = df_WB.loc[df_WB['Series Name'].isin(series_to_keep)]

country_groups = [
    "AFE", "AFW", "ARB", "CSS", "CEB", "EAR", "EAS", "EAP", "TEA", "EMU", 
    "ECS", "ECA", "TEC", "EUU", "FCS", "HPC", "HIC", "IBD", "IBT", "IDB", 
    "IDX", "IDA", "LTE", "LCN", "LAC", "TLA", "LDC", "LMY", "LIC", "LMC", 
    "MEA", "MNA", "TMN", "MIC", "NAC", "INX", "OED", "OSS", "PSS", "PST", 
    "PRE", "SST", "SAS", "TSA", "SSF", "SSA", "TSS", "UMC"]

world = ["WLD"]

filtered_df = df_choice[~df_choice['Country Code'].isin(country_groups + world)]

# download data
#filtered_df.to_csv(cwd + '/data/WB_more_data.csv', index=False)

In [ ]:
filtered_df.sample(5)